## NYC Yellow Line Taxi

#### Importing Pandas and Requests library

In [ ]:
import pandas as pd

#### Import the data

In [ ]:
data = pd.read_csv("nycyellowline.csv")
data

In [ ]:
data.info()

In [ ]:
# Incase the date values are not correctly formatted

# data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'], errors = 'coerce')
# data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'], errors = 'coerce')

In [ ]:
data.info()

#### Drop any duplicates from date columns

In [ ]:
data.drop_duplicates().reset_index(drop=True)
data['trip_id'] = data.index

In [ ]:
data.head(10)

In [ ]:
data.info()

## Creating tables based of the ER Diagram created

#### Creating datetime table.

In [ ]:
datetime = data[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)

datetime['tpep_pickup_datetime'] = data['tpep_pickup_datetime']
datetime['tpep_dropoff_datetime'] = data['tpep_dropoff_datetime']

# Extracting hour, day, month, year, and weekday values in new respective columns
datetime['pickup_hour'] = data['tpep_pickup_datetime'].dt.hour
datetime['pickup_day'] = data['tpep_pickup_datetime'].dt.day
datetime['pickup_month'] = data['tpep_pickup_datetime'].dt.month
datetime['pickup_year'] = data['tpep_pickup_datetime'].dt.year
datetime['pickup_weekday'] = data['tpep_pickup_datetime'].dt.weekday

datetime['tpep_dropoff_datetime'] = data['tpep_dropoff_datetime']

# Extracting hour, day, month, year, and weekday values in new respective columns
datetime['drop_hour'] = data['tpep_dropoff_datetime'].dt.hour
datetime['drop_day'] = data['tpep_dropoff_datetime'].dt.day
datetime['drop_month'] = data['tpep_dropoff_datetime'].dt.month
datetime['drop_year'] = data['tpep_dropoff_datetime'].dt.year
datetime['drop_weekday'] = data['tpep_dropoff_datetime'].dt.weekday

datetime['datetime_id'] = datetime.index

# Combining all the cloumns in a single table 'datetime'
datetime = datetime[['datetime_id', 'tpep_pickup_datetime', 'pickup_hour', 'pickup_day', 'pickup_month', 'pickup_year', 
                     'pickup_weekday', 'tpep_dropoff_datetime', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]


# saving the output to csv
datetime.to_csv('datetime.csv')

#### Creating passenger_count table

In [ ]:
passenger_count_dim = data[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

# saving the output to csv
passenger_count_dim.to_csv('passenger_count.csv')

In [ ]:
data.info()

#### Creating trip_distance table

In [ ]:
trip_distance_dim = data[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]

# saving the output to csv
trip_distance_dim.to_csv('trip_distance.csv')

#### Creating pickup_location table

In [ ]:
pickup_location = data[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location['pickup_location_id'] = pickup_location.index
pickup_location = pickup_location[['pickup_location_id', 'pickup_longitude', 'pickup_latitude']]

# saving the output to csv
pickup_location.to_csv('pickup_location.csv')

#### Creating dropoff_location table

In [ ]:
dropoff_location = data[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location['dropoff_location_id'] = dropoff_location.index
dropoff_location = dropoff_location[['dropoff_location_id', 'dropoff_longitude', 'dropoff_latitude']]

# saving the output to csv
dropoff_location.to_csv('dropoff_location.csv')

#### Making payment_type table in refrence to 'data_dictionary_trip_records_green' file

In [ ]:
payment_type_name = {1:'Credit Card', 2:'Cash', 3:'No Charge', 4:'Dispute', 5:'Unknown', 6:'Voided Trip'}

payment_type = data[['payment_type']].reset_index(drop=True)
payment_type['payment_type_id'] = payment_type.index
payment_type['payment_type_name'] = payment_type['payment_type'].map(payment_type_name)
payment_type = payment_type[['payment_type_id','payment_type','payment_type_name']]

# saving the output to csv
payment_type.to_csv('payment_type.csv')

#### Making rate_code table in refrence to 'data_dictionary_trip_records_green' file

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code = data[['RatecodeID']].reset_index(drop=True)
rate_code['rate_code_id'] = rate_code.index
rate_code['rate_code_name'] = rate_code['RatecodeID'].map(rate_code_type)
rate_code = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

# saving the output to csv
rate_code.to_csv('rate_code.csv')

In [ ]:
data.info()

#### Creating fact_table table

In [90]:
# List of dataframes to be merged
merge_dataframes = [passenger_count_dim, trip_distance_dim, rate_code, pickup_location,
                    dropoff_location, datetime, payment_type]

# Merge dataframes one by one on 'trip_id'
fact_table = data.copy()

for df in merge_dataframes:
    fact_table = fact_table.merge(df, left_on='trip_id', right_on=f'{df.columns[0]}')

# Select desired columns
selected_columns = ['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
                    'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
                    'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
                    'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                    'improvement_surcharge', 'total_amount']

fact_table = fact_table[selected_columns]
fact_table.to_csv('fact_table.csv')